In [2]:
%reload_ext tensorboard
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import time
import os

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

In [4]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [5]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

In [6]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize =[1, 2, 2, 1], strides =[1, 2, 2, 1], padding = 'SAME')

In [7]:
def conv_layer(input, shape):
    W = weight_variable(shape)
    variable_summaries(W) 
    b = bias_variable([shape[3]])
    variable_summaries(b)
    return tf.nn.relu(conv2d(input, W) + b)

In [8]:
def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    variable_summaries(W)
    b = bias_variable([size])
    variable_summaries(b)
    return tf.matmul(input, W) + b

In [9]:
# This helper function , taken from the official TensorFlow documentation ,
# simply adds some ops that take care of logging summaries
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [11]:
LOG_DIR = "logs/summaries"
DIR = "data"
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape = [None, 784])
y_ = tf.placeholder(tf.float32, shape = [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(x, [-1, 28, 28, 1])

with tf.name_scope('conv_1'):
    conv1 = conv_layer(x_image, shape = [5, 5, 1, 32])
    conv1_pool = max_pool_2x2(conv1)
with tf.name_scope('conv_2'):
    conv2 = conv_layer(conv1_pool, shape = [5, 5, 32, 64])
    conv2_pool = max_pool_2x2(conv2)
    conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
with tf.name_scope('full_1'):
    full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))
with tf.name_scope('dropout'):
    full1_drop = tf.nn.dropout(full_1, keep_prob = keep_prob)
with tf.name_scope('activations'):
    y_conv = full_layer(full1_drop, 10)
    variable_summaries(y_conv)

mnist = input_data.read_data_sets('tmp\data', one_hot = True)

with tf.name_scope('cross'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y_))
    tf.summary.scalar('cross_entropy', cross_entropy )

# # SGD
# AdaGrad
train_step = tf.train.AdagradOptimizer(learning_rate = 1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries
merged = tf.summary.merge_all()

Extracting tmp\data\train-images-idx3-ubyte.gz
Extracting tmp\data\train-labels-idx1-ubyte.gz
Extracting tmp\data\t10k-images-idx3-ubyte.gz
Extracting tmp\data\t10k-labels-idx1-ubyte.gz


In [12]:
saver = tf.train.Saver(max_to_keep = 7, keep_checkpoint_every_n_hours = 1)

In [13]:
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train', graph = tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test', graph = tf.get_default_graph())

    sess.run(tf.global_variables_initializer())
    start_time = time.time()

    for i in range (50000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            summary, train_accuracy = sess.run([merged, accuracy], feed_dict = {x:batch[0], y_:batch[1], keep_prob:1.0})
            print ("time {}, step {}, training accuracy {}".format(time.time() - start_time, i, train_accuracy))
            # Add to summaries
            train_writer.add_summary(summary, i)
        sess.run(train_step, feed_dict = {x:batch[0], y_:batch[1], keep_prob:0.5})
        
    saver.save(sess, os.path.join(DIR, "model_ckpt"),global_step = i)
        
    X = mnist.test.images.reshape(10, 1000, 784)
    Y = mnist.test.labels.reshape(10, 1000, 10)
    
    #summary, merged
    test_accuracy = np.mean([sess.run([accuracy], feed_dict = {x:X[i], y_:Y[i], keep_prob:1.0}) for i in range (10)])        
    test_writer.add_summary(summary, i)
    print ("test accuracy: {}".format(test_accuracy))

time 0.6423897743225098, step 0, training accuracy 0.05999999865889549
time 10.278892040252686, step 100, training accuracy 0.2199999988079071
time 19.53389048576355, step 200, training accuracy 0.2800000011920929
time 28.521857023239136, step 300, training accuracy 0.2199999988079071
time 37.95385265350342, step 400, training accuracy 0.2800000011920929
time 47.34743070602417, step 500, training accuracy 0.3799999952316284
time 56.53048253059387, step 600, training accuracy 0.41999998688697815
time 66.1081793308258, step 700, training accuracy 0.36000001430511475
time 75.39254379272461, step 800, training accuracy 0.46000000834465027
time 84.2879741191864, step 900, training accuracy 0.6000000238418579
time 93.65269804000854, step 1000, training accuracy 0.6600000262260437
time 103.00643253326416, step 1100, training accuracy 0.6000000238418579
time 111.56374597549438, step 1200, training accuracy 0.699999988079071
time 121.21446013450623, step 1300, training accuracy 0.63999998569488

time 1164.0869240760803, step 11400, training accuracy 0.8999999761581421
time 1173.8198206424713, step 11500, training accuracy 0.9399999976158142
time 1183.360641002655, step 11600, training accuracy 0.8999999761581421
time 1192.858847618103, step 11700, training accuracy 0.9399999976158142
time 1202.4555780887604, step 11800, training accuracy 0.8399999737739563
time 1211.3019502162933, step 11900, training accuracy 0.8799999952316284
time 1220.324697971344, step 12000, training accuracy 0.8600000143051147
time 1229.87251329422, step 12100, training accuracy 0.8999999761581421
time 1239.6656250953674, step 12200, training accuracy 0.8799999952316284
time 1249.2782900333405, step 12300, training accuracy 0.8799999952316284
time 1258.9489724636078, step 12400, training accuracy 0.8600000143051147
time 1268.6142513751984, step 12500, training accuracy 0.8999999761581421
time 1278.131920337677, step 12600, training accuracy 0.8999999761581421
time 1287.5009260177612, step 12700, trainin

time 2249.2149410247803, step 22600, training accuracy 0.8199999928474426
time 2258.9536623954773, step 22700, training accuracy 0.9399999976158142
time 2282.1691749095917, step 22800, training accuracy 0.9399999976158142
time 2288.99720287323, step 22900, training accuracy 0.7400000095367432
time 2294.4525480270386, step 23000, training accuracy 0.9399999976158142
time 2299.9938201904297, step 23100, training accuracy 0.8799999952316284
time 2305.0686752796173, step 23200, training accuracy 0.9800000190734863
time 2310.8099024295807, step 23300, training accuracy 0.8399999737739563
time 2317.094401359558, step 23400, training accuracy 0.9399999976158142
time 2323.545953273773, step 23500, training accuracy 0.8399999737739563
time 2328.984482049942, step 23600, training accuracy 0.9200000166893005
time 2334.7001700401306, step 23700, training accuracy 0.8999999761581421
time 2340.108699321747, step 23800, training accuracy 0.8600000143051147
time 2345.533193588257, step 23900, training

time 2979.2948083877563, step 33800, training accuracy 0.9800000190734863
time 2985.9946854114532, step 33900, training accuracy 0.8999999761581421
time 2992.1350014209747, step 34000, training accuracy 0.8799999952316284
time 2998.4930353164673, step 34100, training accuracy 0.9399999976158142
time 3005.0000805854797, step 34200, training accuracy 0.9200000166893005
time 3011.271208047867, step 34300, training accuracy 0.9800000190734863
time 3017.183465719223, step 34400, training accuracy 0.8999999761581421
time 3023.3626754283905, step 34500, training accuracy 0.8999999761581421
time 3029.6358966827393, step 34600, training accuracy 0.8999999761581421
time 3035.8771934509277, step 34700, training accuracy 0.9800000190734863
time 3042.3260927200317, step 34800, training accuracy 0.9399999976158142
time 3048.562439441681, step 34900, training accuracy 0.8999999761581421
time 3055.2438127994537, step 35000, training accuracy 0.9200000166893005
time 3061.7675614356995, step 35100, trai

time 3721.6542053222656, step 45100, training accuracy 0.9599999785423279
time 3728.8455486297607, step 45200, training accuracy 0.9399999976158142
time 3735.588937997818, step 45300, training accuracy 0.8799999952316284
time 3742.304979801178, step 45400, training accuracy 0.9800000190734863
time 3749.1776025295258, step 45500, training accuracy 0.8600000143051147
time 3755.843806743622, step 45600, training accuracy 0.9399999976158142
time 3762.3384115695953, step 45700, training accuracy 1.0
time 3768.8869013786316, step 45800, training accuracy 0.8999999761581421
time 3775.1531851291656, step 45900, training accuracy 1.0
time 3781.5710620880127, step 46000, training accuracy 1.0
time 3788.0412681102753, step 46100, training accuracy 0.9599999785423279
time 3795.032578229904, step 46200, training accuracy 0.9399999976158142
time 3801.5885937213898, step 46300, training accuracy 0.8999999761581421
time 3808.1480503082275, step 46400, training accuracy 0.9399999976158142
time 3814.578

In [14]:
train_var = [x, y_, keep_prob, accuracy]
tf.add_to_collection('train_var', train_var[0])
tf.add_to_collection('train_var', train_var[1])
tf.add_to_collection('train_var', train_var[2])
tf.add_to_collection('train_var', train_var[3])

saver.export_meta_graph(os.path.join(DIR,"model_ckpt-9.meta"), collection_list = ['train_var'])

meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "AddV2"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
      

In [18]:
tf.reset_default_graph()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    start_time = time.time()
    saver = tf.train.import_meta_graph(os.path.join(DIR,"model_ckpt-9.meta"))
    saver.restore(sess,os.path.join(DIR,"model_ckpt-49999"))
    x = tf.get_collection('train_var')[0]
    y_ = tf.get_collection('train_var')[1]
    keep_prob = tf.get_collection('train_var')[2]
    accuracy = tf.get_collection('train_var')[3]
    
 
    test_accuracy = np.mean([sess.run([accuracy], feed_dict = {x:X[i], y_:Y[i], keep_prob:1.0}) for i in range (10)])
    print("test accuracy: {}".format(test_accuracy))

INFO:tensorflow:Restoring parameters from data\model_ckpt-49999
test accuracy: 0.9359999895095825


In [19]:
%tensorboard --logdir logs/summaries 

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 39128.